In [ ]:
# !pip install tiktoken

Model references
================
https://learn.microsoft.com/en-US/azure/ai-services/openai/reference

Tutorial references
===================
https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=command-line


In [ ]:
import openai
import os
import re
import requests
# import sys
# from num2words import num2words
import os
import pandas as pd
# import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-05-15"

url = openai.api_base + "/openai/deployments?api-version=2023-03-15-preview" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

In [ ]:
df = pd.read_csv("master.csv") # example: df = pd.read_csv("./data/bill_sum_data.csv")df
df

In [ ]:
df_bills = df[['text', 'videoId', 'description', 'title', 'speaker']]
df_bills

In [ ]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text'] = df_bills["text"].apply(lambda x : normalize_text(x))

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

In [ ]:
df_bills

In [ ]:
print(df_bills.text[2])

sample_encode = tokenizer.encode(df_bills.text[2]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

In [ ]:
len(decode)

In [ ]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [ ]:
df_bills

In [ ]:
# Persist the embeddings to disk as a csv file
df_bills.to_csv("master_embeddings.csv", index=False)